# Тестирование обобщающего NN-классификатора новообразований молочной железы

In [1]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import definitions as defs

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
entire_ds = dataframe_to_dataset(df, 'tmp entire.csv')
train_ds = dataframe_to_dataset(train_df, 'tmp train.csv')
val_ds = dataframe_to_dataset(val_df, 'tmp val.csv')
test_ds = dataframe_to_dataset(test_df, 'tmp test.csv')

Построим матрицы ошибок с ещё одной меткой "неопределён". Эта метку будем назначать в том случае, если ни для одной другой метки вероятность не превысила заданный порог. Порог будем менять.

In [ ]:
def get_true_predicted_labels(threshold):
    true_labels = []
    predicted_labels = []
    for point, true_probabilities in test_ds:
        true_label_index = np.array(true_probabilities[0]).argmax()
        true_label = defs.LABELS[true_label_index]
        true_labels.append(true_label)

        predicted_probabilities = best_model.predict(point)
        best_predicted_probability = np.array(predicted_probabilities[0]).max()
        if best_predicted_probability < threshold:
            predicted_label = 'неопределён'
        else:
            predicted_label_index = np.array(predicted_probabilities[0]).argmax()
            predicted_label = defs.LABELS[predicted_label_index]
        predicted_labels.append(predicted_label)
    return true_labels, predicted_labels


def plot_confusion_matrix(threshold):
    true_labels, predicted_labels = get_true_predicted_labels(threshold)
    cm = confusion_matrix(true_labels, predicted_labels)
    ConfusionMatrixDisplay(cm).plot()

In [ ]:
for threshold in [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    plot_confusion_matrix(threshold)

In [ ]:
def test():
    """ Выводит точность классификации со всеми признаками и точности без определённых признаков """
    test_ds_evaluation = best_model.evaluate(test_ds, verbose=0)[1]
    print('Точность с учётом всех признаков: {:2.2%}'.format(test_ds_evaluation))
    
    def zero_feature(point, label):
        """ Зануляет признак """
        if defs.INPUT_NAMES[feature_name] == 'numerical':
            point[feature_name] = tf.constant(
                np.array(0., dtype=np.float32),
                dtype=tf.float32,
                shape=(1,)
            )
        else:
            point[feature_name] = tf.constant(
                np.array(0, dtype=np.int32),
                dtype=tf.int32,
                shape=(1,)
            )        
        return point, label
        
    for feature_name in defs.INPUT_NAMES.keys():
        test_ds_without_feature = test_ds.map(zero_feature)
        test_ds_without_feature_evaluation = best_model.evaluate(test_ds_without_feature, verbose=0)[1]
        change = test_ds_without_feature_evaluation - test_ds_evaluation
        print('Точность без признака \"{}\": {:2.2%}. Изменение: {:2.2%}'.format(
            feature_name,
            test_ds_without_feature_evaluation,
            change
        ))
    
test()

In [ ]:
# # удаляем временные файлы
# os.remove(os.path.join(os.getcwd(), 'data', 'tmp entire.csv'))
# os.remove(os.path.join(os.getcwd(), 'data', 'tmp train.csv'))
# os.remove(os.path.join(os.getcwd(), 'data', 'tmp val.csv'))
# os.remove(os.path.join(os.getcwd(), 'data', 'tmp test.csv'))